In [2]:
from function import *

2022-07-29 10:34:10.288405: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

/home/wmbio/WORK/gitworking/Dependency_prediction/prediction/src
/home/wmbio/WORK/gitworking/Dependency_prediction


## **Data Load & Path**

In [4]:
TRAIN_PATH = "/home/wmbio/WORK/gitworking/Dependency_prediction/preprocessing/DATA/2022-07-28/"
TEMP_PATH = "prediction/train_preprocessing/"
SAVE_PATH = "prediction/custom_model/"

In [ ]:
def trainset_load(npz_path="prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples.npz", 
                  filename="training_custom"):
    
    if exists(npz_path) is False:
        # expression
        if exists(TEMP_PATH + filename + "_exp_training.npy") is False:
            data_exp, sample_names_exp, property_names_exp = load_data(TRAIN_PATH + filename + "_exp_training.txt")
            np.save(TEMP_PATH + filename + "_exp_training.npy", data_exp)
        else:
            data_exp = np.load(TEMP_PATH + filename + "_exp_training.npy")
        print("exp load.. completed")    

        # mutation
        if exists(TEMP_PATH + filename + "_mut_training.npy") is False:
            data_mut, sample_names_mut, property_names_mut = load_data(TRAIN_PATH + filename + "_mut_training.txt")
            np.save(TEMP_PATH + filename + "_mut_training.npy", data_mut)
        else:
            data_mut = np.load(TEMP_PATH + filename + "_mut_training.npy")
        print("mut load.. completed")

        # copy number alteration
        if exists(TEMP_PATH + filename + "_cna_training.npy") is False:
            data_cna, sample_names_cna, property_names_cna = load_data(TRAIN_PATH + filename + "_cna_training.txt")
            np.save(TEMP_PATH + filename + "_cna_training.npy", data_cna)
        else:
            data_cna = np.load(TEMP_PATH + filename + "_cna_training.npy")
        print("cna load.. completed")

        # methylation
        if exists(TEMP_PATH + filename + "_meth_training.npy") is False:
            data_meth, sample_names_meth, property_names_meth = load_data(TRAIN_PATH + filename + "_meth_training.txt")
            np.save(TEMP_PATH + filename + "_meth_training.npy", data_meth)
        else:
            data_meth = np.load(TEMP_PATH + filename + "_meth_training.npy")
        print("meth load.. completed")

        # dependency score
        if exists(TEMP_PATH + filename + "_DepScore_training.npy") is False:
            data_dep, sample_names_dep, property_names_dep = load_data(TRAIN_PATH + filename + "_DepScore_training.txt")
            np.save(TEMP_PATH + filename + "_DepScore_training.npy", data_dep)
        else:
            data_dep = np.load(TEMP_PATH + filename + "_DepScore_training.npy")
        print("dep load.. completed")

        # fingerprint
        if exists(TEMP_PATH + filename + "_fingerprint_training.npy") is False:
            data_fprint, sample_names_fprint, property_names_fprint = load_data(TRAIN_PATH + filename + "_fingerprint_training.txt")
            np.save(TEMP_PATH + filename + "_fingerprint_training.npy", data_fprint)
        else:
            data_fprint = np.load(TEMP_PATH + filename + "_fingerprint_training.npy")
        print("fingerprint load.. completed")

        np.savez_compressed(npz_path, 
                            data_exp=data_exp, data_mut=data_mut, data_cna=data_cna, data_meth=data_meth,
                            data_dep=data_dep, data_fprint=data_fprint)
    else :
        data = np.load(npz_path)
        data_exp = data['data_exp']
        data_mut = data['data_mut']
        data_cna = data['data_cna']
        data_meth = data['data_meth']
        data_fprint = data['data_fprint']
        data_dep = data['data_dep']
        
    return data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep

In [6]:
data_cna, sample_names_cna, property_names_cna = load_data(TRAIN_PATH + training_custom + "_cna_training.txt")

NameError: name 'filename' is not defined

In [7]:
df_dt = dt.fread(TRAIN_PATH + "training_custom_cna_training.txt", sep="\t")

In [8]:
df_dt

,Bin,C1G1,C1G2,C1G3,C1G4,C1G5,C1G6,C1G7,C1G8,C1G9,…,C501G1223,C501G1224,C501G1225,C501G1226,C501G1227
,▪▪▪▪,,,,,,,,,,,,,,,
0,chr7_5473to5474_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
1,chr7_5474to5475_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
2,chr7_5475to5476_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
3,chr7_5476to5477_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
4,chr7_5477to5478_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
5,chr7_5478to5479_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
6,chr7_5479to5480_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
7,chr7_5480to5481_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)
8,chr7_5481to5482_10k,(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),(unknown),…,(unknown),(unknown),(unknown),(unknown),(unknown)


In [ ]:
gene_names = df_dt[0].to_list()[0]

In [ ]:
sample_names = list(df_dt.names[1:])

In [ ]:
data_np = df_dt[:, 1:].to_numpy()

In [ ]:
data_np = np.transpose(data_np)

In [ ]:
gc.collect()

In [5]:
np.load(TEMP_PATH + 'training_custom_cna_training.npy')

array([[b'', b'', b'', ..., b'', b'', b''],
       [b'', b'', b'', ..., b'', b'', b''],
       [b'', b'', b'', ..., b'', b'', b''],
       ...,
       [b'', b'', b'', ..., b'', b'', b''],
       [b'', b'', b'', ..., b'', b'', b''],
       [b'', b'', b'', ..., b'', b'', b'']], dtype='|V0')

In [ ]:
if exists("prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples.npz") is False:
    # expression
    if exists(TEMP_PATH + "training_custom_exp_training.npy") is False:
        data_exp, sample_names_exp, property_names_exp = load_data(TRAIN_PATH + "training_custom_exp_training.txt")
        np.save(TEMP_PATH + 'training_custom_exp_training.npy', data_exp)
    else:
        data_exp = np.load(TEMP_PATH + 'training_custom_exp_training.npy')
    print("exp load.. completed")    
    
    # mutation
    if exists(TEMP_PATH + "training_custom_mut_training.npy") is False:
        data_mut, sample_names_mut, property_names_mut = load_data(TRAIN_PATH + "training_custom_mut_training.txt")
        np.save(TEMP_PATH + 'training_custom_mut_training.npy', data_mut)
    else:
        data_mut = np.load(TEMP_PATH + 'training_custom_exp_training.npy')
    print("mut load.. completed")
    
    # copy number alteration
    if exists(TEMP_PATH + "training_custom_cna_training.npy") is False:
        data_cna, sample_names_cna, property_names_cna = load_data(TRAIN_PATH + "training_custom_cna_training.txt")
        np.save(TEMP_PATH + 'training_custom_cna_training.npy', data_cna)
    else:
        np.load(TEMP_PATH + 'training_custom_cna_training.npy')
    print("cna load.. completed")
    
    # methylation
    if exists(TEMP_PATH + "training_custom_meth_training.npy") is False:
        data_meth, sample_names_meth, property_names_meth = load_data(TRAIN_PATH + "training_custom_meth_training.txt")
        np.save(TEMP_PATH + 'training_custom_meth_training.npy', data_meth)
    else:
        data_meth = np.load(TEMP_PATH + 'training_custom_meth_training.npy')
    print("meth load.. completed")
    
    # dependency score
    if exists(TEMP_PATH + "training_custom_DepScore_training.npy") is False:
        data_dep, sample_names_dep, property_names_dep = load_data(TRAIN_PATH + "training_custom_DepScore_training.txt")
        np.save(TEMP_PATH + 'training_custom_DepScore_training.npy', data_dep)
    else:
        data_dep = np.load(TEMP_PATH + 'training_custom_DepScore_training.npy')
    print("dep load.. completed")
    
    # fingerprint
    if exists(TEMP_PATH + "training_custom_fingerprint_training.npy") is False:
        data_fprint, sample_names_fprint, property_names_fprint = load_data(TRAIN_PATH + "training_custom_fingerprint_training.txt")
        np.save(TEMP_PATH + 'training_custom_fingerprint_training.npy', data_fprint)
    else:
        data_fprint = np.load(TEMP_PATH + 'training_custom_fingerprint_training.npy')
    print("fingerprint load.. completed")
        
    np.savez_compressed("prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples_custom.npz", 
                        data_exp=data_exp, data_mut=data_mut, data_cna=data_cna, data_meth=data_meth,
                        data_dep=data_dep, data_fprint=data_fprint)
else :
    data = np.load("prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples_custom.npz")
    data_exp = data['data_exp']
    data_mut = data['data_mut']
    data_cna = data['data_cna']
    data_meth = data['data_meth']
    data_fprint = data['data_fprint']
    data_dep = data['data_dep']

## **Build Model**

* **Pretrained Model load - TCGA**

In [ ]:
premodel_mut = pickle.load(open('prediction/data/premodel_tcga_mut_1000_100_50.pickle', 'rb'))
premodel_exp = pickle.load(open('prediction/data/premodel_tcga_exp_500_200_50.pickle', 'rb'))
premodel_cna = pickle.load(open('prediction/data/premodel_tcga_cna_500_200_50.pickle', 'rb'))
premodel_meth = pickle.load(open('prediction/data/premodel_tcga_meth_500_200_50.pickle', 'rb'))
print("\n\nDatasets successfully loaded.")

In [ ]:
activation_func = 'relu' # for all middle layers
activation_func2 = 'linear' # for output layer to output unbounded gene-effect scores
init = 'he_uniform'
dense_layer_dim = 250
batch_size = 500
num_epoch = 100
num_DepOI = 1298 # 1298 DepOIs as defined in our paper, custom 1227
num_ccl = int(data_mut.shape[0]/num_DepOI)
split_ratio = 0.7

In [ ]:
# 80% CCLs for training/validation, and 20% for testing
id_rand = np.random.permutation(num_ccl)
id_cell_train = id_rand[np.arange(0, round(num_ccl * split_ratio))]
id_cell_test = id_rand[np.arange(round(num_ccl * split_ratio), num_ccl)]
id_train = np.arange(0, num_DepOI) + id_cell_train[0]*num_DepOI
for y in id_cell_train:
    id_train = np.union1d(id_train, np.arange(0, num_DepOI) + y*num_DepOI)
id_test = np.arange(0, num_DepOI) + id_cell_test[0] * num_DepOI
for y in id_cell_test:
    id_test = np.union1d(id_test, np.arange(0, num_DepOI) + y*num_DepOI)
print("\n\nTraining/validation on %d samples (%d CCLs x %d DepOIs) and testing on %d samples (%d CCLs x %d DepOIs).\n\n" % (
    len(id_train), len(id_cell_train), num_DepOI, len(id_test), len(id_cell_test), num_DepOI))

## **Fit & Evaluation Model**

### **(1) Full Model 4-Omics**

In [ ]:
train_generator = generator(data_mut[id_train], data_exp[id_train], data_cna[id_train], data_meth[id_train], 
                                    data_fprint[id_train], data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator(data_mut[id_test], data_exp[id_test], data_cna[id_test], data_meth[id_test], 
                                    data_fprint[id_test], data_dep[id_test], batch_size = batch_size)

In [ ]:
model = full_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
SAVE_NAME = "model_custom_full_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

In [ ]:
model_train_vis(history=history)

### **(2) Expression-Mutation-CNA Model**

In [ ]:
train_generator = generator3(data_mut[id_train], data_exp[id_train], data_cna[id_train], 
                                    data_fprint[id_train], data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator3(data_mut[id_test], data_exp[id_test], data_cna[id_test],
                                    data_fprint[id_test], data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_mut_cna_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
SAVE_NAME = "model_custom_mut_exp_cna_0725"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

In [ ]:
model_train_vis(history=history)

### **(3) Expression-Mutation Model**

In [ ]:
train_generator = generator2(data_mut[id_train], data_exp[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator2(data_mut[id_test], data_exp[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_mut_exp_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(4) Expression Model**

In [ ]:
train_generator = generator1(data_exp[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_exp[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
with tf.device('/cpu:0'):
    model = exp_model()
    tf.keras.utils.plot_model(model, show_shapes=True,  show_dtype=True, to_file='model_custom.png')
    es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
    model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
    history = model.fit(train_generator, 
                        validation_data=test_generator, 
                        # epochs=num_epoch, 
                        epochs=100, 
                        verbose=1,         
                        callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_ccl501_exp_0728"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(5) Mutation Model**

In [ ]:
train_generator = generator1(data_mut[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_mut[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_mut_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(6) CNA Model**

In [ ]:
train_generator = generator1(data_cna[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_cna[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_cna_0726"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))